In [2]:
%matplotlib inline 
from datetime import datetime 
from random import randrange 
import random
import operator 
import pandas as pd 
import matplotlib.pyplot as plt 
import re 
import numpy as np
import collections

In [3]:
df = pd.read_csv('capture20110811.pcap.netflow.labeled', sep ='\s+')
df.head()

,#Date_flow,start,Durat,Prot,Src_IP_Addr:Port,->,Dst_IP_Addr:Port,Flags,Tos,Packets,Bytes,Flows,Label(LEGITIMATE:Botnet:Background)
0,2011-08-11,10:10:00.003,0.967,UDP,89.31.8.11:23929,->,147.32.84.229:13363,INT,0,2,135,1,Background
1,2011-08-11,10:10:00.003,0.967,UDP,147.32.84.229:13363,->,89.31.8.11:23929,INT,0,2,276,1,Background
2,2011-08-11,10:10:00.006,0.000,UDP,208.88.186.6:34042,->,147.32.84.229:13363,INT,0,1,62,1,Background
3,2011-08-11,10:10:00.008,0.000,UDP,92.118.218.77:55246,->,147.32.84.229:13363,INT,0,1,78,1,Background
4,2011-08-11,10:10:00.009,0.000,UDP,182.185.139.181:10223,->,147.32.84.229:13363,INT,0,1,72,1,Background


In [4]:
df[['Src_IP_Addr', 'Src_Port']] = df['Src_IP_Addr:Port'].str.split(':', n=1, expand=True)
df[['Dst_IP_Addr', 'Dst_Port']] = df['Dst_IP_Addr:Port'].str.split(':', n=1, expand=True)
#df['Date']= df['#Date_flow'].map(str)+ '' + df['start']
#df['Date']= pd.to_datetime(df['Date'], format = '%Y-%m-%d %H:%M:%S.%f', exact=True)
df['Label'] = df['Label(LEGITIMATE:Botnet:Background)']

df = df.drop(labels=['->', 'Label(LEGITIMATE:Botnet:Background)', 'Src_IP_Addr:Port', 'Dst_IP_Addr:Port'], axis=1)

print(df.shape)
print(df.columns)
df.head()

(6351188, 14)
Index(['#Date_flow', 'start', 'Durat', 'Prot', 'Flags', 'Tos', 'Packets',
       'Bytes', 'Flows', 'Src_IP_Addr', 'Src_Port', 'Dst_IP_Addr', 'Dst_Port',
       'Label'],
      dtype='object')


,#Date_flow,start,Durat,Prot,Flags,Tos,Packets,Bytes,Flows,Src_IP_Addr,Src_Port,Dst_IP_Addr,Dst_Port,Label
0,2011-08-11,10:10:00.003,0.967,UDP,INT,0,2,135,1,89.31.8.11,23929,147.32.84.229,13363,Background
1,2011-08-11,10:10:00.003,0.967,UDP,INT,0,2,276,1,147.32.84.229,13363,89.31.8.11,23929,Background
2,2011-08-11,10:10:00.006,0.000,UDP,INT,0,1,62,1,208.88.186.6,34042,147.32.84.229,13363,Background
3,2011-08-11,10:10:00.008,0.000,UDP,INT,0,1,78,1,92.118.218.77,55246,147.32.84.229,13363,Background
4,2011-08-11,10:10:00.009,0.000,UDP,INT,0,1,72,1,182.185.139.181,10223,147.32.84.229,13363,Background


In [5]:
print(df['Src_IP_Addr'].value_counts()[0:10])
infected_IP = '147.32.84.165'
val = df['Src_IP_Addr'].value_counts()[infected_IP] + df['Dst_IP_Addr'].value_counts()[infected_IP]
print("")
print("Infected ip occurrences: ", val)
# top 10 freq


147.32.84.229    1458791
147.32.80.9       573079
147.32.84.59      347413
147.32.84.138     331209
147.32.86.165      82589
147.32.84.165      45261
147.32.84.171      34186
147.32.84.118      34027
147.32.85.25       31279
147.32.86.20       28073
Name: Src_IP_Addr, dtype: int64

Infected ip occurrences:  54433


In [28]:
inf = df.loc[(df['Src_IP_Addr']== infected_IP)]
print(inf.shape)
inf.head()

(45261, 14)


,#Date_flow,start,Durat,Prot,Flags,Tos,Packets,Bytes,Flows,Src_IP_Addr,Src_Port,Dst_IP_Addr,Dst_Port,Label
540283,2011-08-11,10:27:20.087,0.000,UDP,INT,0,1,64,1,147.32.84.165,1025,147.32.80.9,53,Botnet
541362,2011-08-11,10:27:22.334,0.000,UDP,INT,0,1,87,1,147.32.84.165,1025,147.32.80.9,53,Botnet
541377,2011-08-11,10:27:22.355,0.045,TCP,SRPA_,0,4,629,1,147.32.84.165,1027,74.125.232.198,80,Botnet
702906,2011-08-11,10:32:25.092,0.000,UDP,INT,0,1,78,1,147.32.84.165,1025,147.32.80.9,53,Botnet
703012,2011-08-11,10:32:25.294,0.126,TCP,SPA_,0,6,471,1,147.32.84.165,1039,195.113.232.98,80,Botnet


In [29]:
def common_interactions(data,infected):
    
    ips={}
    for index, row in data.iterrows():
            if row["Src_IP_Addr"]==infected:
                if row["Dst_IP_Addr"] in ips:
                    ips[row["Dst_IP_Addr"]]+=1
                else:
                    ips[row["Dst_IP_Addr"]]=1
            else:
                if row["Src_IP_Addr"] in ips:
                    ips[row["Src_IP_Addr"]]+=1
                else:
                    ips[
    ips = sorted(ips.items(), key=operow["Src_IP_Addr"]]=1   rator.itemgetter(1), reverse= True)
    top = ips[0:10]
    
    return top, ips   


In [31]:
top_10 , ips = common_interactions(inf, infected_IP)

print (np.array(top_10))


[['193.23.181.44' '6167']
 ['174.128.246.102' '3427']
 ['174.37.196.55' '3357']
 ['67.19.72.206' '3135']
 ['72.20.15.61' '2966']
 ['173.236.31.226' '1708']
 ['184.154.89.154' '1678']
 ['46.4.36.120' '1627']
 ['147.32.80.9' '787']
 ['217.163.21.37' '701']]


In [32]:
k=[10,50,100,500,1000,5000,10000]


In [37]:
def sample(data, k):
    reservoir = []
    for t, item in enumerate(data['Dst_IP_Addr']):
        if t < k:
            reservoir.append(item)
        else:
            m = random.randint(0,1)
            if m > k/t:
             # random replace n item 
                ind = np.random.randint(0,k)
                reservoir[ind] = item
    return reservoir

In [38]:
s1 = sample(inf, 10000)
count= collections.Counter(s1).most_common(10)
print(count)

[('193.23.181.44', 891), ('174.128.246.102', 529), ('184.154.89.154', 515), ('174.37.196.55', 495), ('46.4.36.120', 461), ('173.236.31.226', 440), ('72.20.15.61', 415), ('67.19.72.206', 407), ('147.32.80.9', 203), ('74.6.136.244', 176)]


In [39]:
s1 = sample(inf, 1000)
count1= collections.Counter(s1).most_common(10)
print(count1)

[('184.154.89.154', 81), ('46.4.36.120', 59), ('98.139.175.225', 29), ('184.82.148.43', 24), ('98.137.54.237', 24), ('217.163.21.39', 24), ('147.32.80.9', 22), ('74.6.136.244', 22), ('94.100.28.114', 21), ('209.85.143.27', 20)]


In [40]:
s2 = sample (inf,10000)
count2= collections.Counter(s2).most_common(10)
print(count2)

[('193.23.181.44', 878), ('184.154.89.154', 554), ('174.128.246.102', 508), ('174.37.196.55', 500), ('67.19.72.206', 487), ('173.236.31.226', 450), ('46.4.36.120', 441), ('72.20.15.61', 434), ('147.32.80.9', 190), ('74.6.136.244', 176)]


In [41]:
for i in k:
    s = sample(inf, i)
    count= collections.Counter(s).most_common(10)
    print('For k:', i)
    print('')
    print(count)
    print('')

For k: 10

[('92.240.244.181', 2), ('46.4.36.120', 1), ('66.94.237.64', 1), ('217.163.21.35', 1), ('76.96.30.116', 1), ('205.188.103.2', 1), ('64.12.138.161', 1), ('184.154.89.154', 1), ('217.163.21.39', 1)]

For k: 50

[('217.163.21.39', 8), ('173.241.240.4', 3), ('46.4.36.120', 3), ('184.82.147.252', 3), ('184.154.89.154', 3), ('76.96.30.116', 2), ('64.12.138.161', 2), ('98.139.175.225', 2), ('98.137.54.237', 2), ('209.85.143.27', 2)]

For k: 100

[('184.154.89.154', 8), ('98.137.54.237', 8), ('46.4.36.120', 6), ('209.85.143.27', 6), ('98.139.175.225', 5), ('92.240.244.181', 4), ('217.163.21.39', 4), ('184.82.147.252', 4), ('184.82.147.251', 4), ('173.241.240.4', 3)]

For k: 500

[('184.154.89.154', 45), ('217.163.21.39', 27), ('46.4.36.120', 27), ('98.137.54.237', 24), ('98.139.175.225', 19), ('147.32.80.9', 14), ('209.85.143.27', 14), ('184.82.148.43', 12), ('217.163.21.37', 12), ('184.82.148.44', 11)]

For k: 1000

[('184.154.89.154', 90), ('46.4.36.120', 56), ('184.82.148.43', 32

In [12]:
#def reservoir_sampling(items, k):
    """ 
    Reservoir sampling algorithm for large sample space or unknow end list
    See <http://en.wikipedia.org/wiki/Reservoir_sampling> for detail>
    Type: ([a] * Int) -> [a]
    Prev constrain: k is positive and items at least of k items
    Post constrain: the length of return array is k
    """
    #sample = items[0:k]

   # for i in range(k, len(items)):
    #    j = randrange(1, i + 1)
     #   if j < k:
            sample[j] = items[i]

   # return sample

In [15]:
#reservoir_sampling(np.arange(len(inf),dtype= int), 100)

array([    0, 47564, 41762, 50904, 47883, 23266, 27001, 23012, 32362,
        5873,  3296, 36442, 52566, 13802, 23683, 20954, 27212, 25652,
       26205,  4448,  7770,  9423, 12594, 53373, 38177,  4916, 14578,
       53664, 44409, 19790, 41904, 24850, 26595, 25766, 30430, 30606,
       44590,  8537, 27505, 38925, 41766, 35028, 29097, 35792, 49577,
       15940, 31070, 38850, 19057, 18115, 52981, 49258, 44355, 52142,
       52752, 24945, 13853, 43469, 44368,  3834,  8992, 38624, 50667,
        1375,  2181, 28582,  7912,  7080, 13320, 19586, 25590, 14998,
       12603, 33085,  6917,  5561, 45357, 16046, 27591, 22215,  6200,
       51426,  9872,  8749,  5347, 16156, 43553, 53695,  6582, 16750,
        5803, 47632, 50596, 16572, 13736, 29163, 23927,  8229, 36044,
       30174])